In [1]:
import requests 
import bs4 
import pandas as pd
import re
#import urllib
from requests.exceptions import MissingSchema

In [2]:
def gen_add(sp):
    raw = sp.findAll(['div', 'a'], 
                   {"class": ['person-list-small__name', 
                              'person-list-small__twitter-handle']})
    desc_raw = sp.findAll('p', {"class": ['person-list-small__title']})
    
    names = []
    dat = [i.text for i in raw]
    
    while(len(dat)!=0):
        if (len(dat) == 1) or (dat[1][0] != '@' and len(dat[1].split(' '))>1):
            names.append((dat[0], None))
            dat.remove(dat[0])
        else:
            names.append((dat[0], dat[1]))
            dat.remove(dat[0])
            dat.remove(dat[0])
            
    add = pd.DataFrame({'name': [i[0] for i in names],
                        'twitter': [i[1] for i in names],
                        'desc': [i.text for i in desc_raw]})
    return add

In [4]:
user_data = pd.DataFrame()

for ch in range(ord('A'), ord('Z') + 1):
    print(chr(ch), end = '\r')
    url = f'https://www.heritage.org/about-heritage/staff/experts/{chr(ch)}'
    r = requests.get(url)
    soup = bs4.BeautifulSoup(r.text, "html.parser").body
    add = gen_add(soup)
    user_data = user_data.append(add)

In [6]:
for ch in range(ord('A'), ord('Z') + 1):
    print(chr(ch), end = '\r')
    url = f'https://www.heritage.org/about-heritage/staff/leadership/{chr(ch)}'
    r = requests.get(url)
    soup = bs4.BeautifulSoup(r.text, "html.parser").body
    add = gen_add(soup)
    user_data = user_data.append(add)

In [7]:
user_data = user_data.drop_duplicates().copy()

In [8]:
user_data['twitter'] = user_data['twitter'].apply(lambda x: x.strip('@') if x != None else x)
user_data.to_csv('heritage_twitter.csv', index=False)